In [16]:
from jupyter_plotly_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash
from dash.dependencies import Input, Output
from pymongo import MongoClient
import urllib.parse
from bson.json_util import dumps
import dash_leaflet as dl
import plotly.express as px
import dash_table as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


app = JupyterDash('Grazioso Salvare')
###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Skywalker7"
#Initializing the database connection
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}), index='_id')


image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #unique identifier
    html.Center(html.B(html.H1('Ritesh Shah SNHU CS-340 Dashboard'))),
    #Show Grazioso logo
    html.A(href="https://www.snhu.edu",children=[html.Img(alt="Link to home page", src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height':'20%', 'width':'20%', 'padding':'auto'})]),
    html.Hr(),
    #add radio buttons for use in data filtering
    html.Div(dcc.RadioItems(
                id='radio-items',
                options = [
                    {'label': 'Water Rescue', 'value': '1'},
                    {'label': 'Mountain Rescue', 'value': '2'},
                    {'label': 'Disaster Rescue', 'value': '3'},
                    {'label': 'Reset', 'value': '4'},

                    ],
                value = '4',
                labelStyle={'display': 'inline-block'}
    )), 
    #add datatable for displaying the database contents
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        filter_action ="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        

    ),
    html.Br(),
     html.Hr(),
    #add div to hold the graph and map on the same line
     html.Div(className='row',
         style={'display':'inline-flex', 'width':'100%', 'border':'1px solid #000000', 'overflow':'hidden'},
             children=[
                 #adding div containers to hold graph and map and style them for presentation
                html.Div(id='graph-id', className='col s12 m6', style={'width':'60%','height':'700px', 'border-right':'1px solid #000000'}),
                html.Div(id='map-id', className='col s12 m6', style={'width':'40%','height':'700px'})
            ]
        )
]
)

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




@app.callback(Output('datatable-id','data'),
    [Input('radio-items', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
# use higher number of button clicks to determine filter type, can you think of a better way ? ....
# Yes. Using the radio-items value
    if (int(filter_type) == 4):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}), index='_id')
    elif (int(filter_type) == 1):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}), index='_id')
    elif (int(filter_type) == 2):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}), index='_id')
    elif (int(filter_type) == 3):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}), index='_id')
        
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame.from_dict(viewData)
    
    #pie chart
    fig_hist1 = px.pie(df, values='age_upon_outcome_in_weeks', names='age_upon_outcome_in_weeks', color='age_upon_outcome_in_weeks', title='Available dogs by age in weeks')

    #scatter graph
    fig_hist2 = px.scatter(df, x='breed', y="age_upon_outcome_in_weeks", color='breed', title='Available breeds by age')
    #bar graph
    #ig_hist2 = px.histogram(df, x='sex_upon_outcome', color='sex_upon_outcome', title='Available breeds by sex')
    return [
        dcc.Graph(figure = fig_hist1, style={'width':'100%','height':'50%'}),
        dcc.Graph(figure = fig_hist2, style={'width':'100%','height':'50%'})
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    df = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(center=[df.loc[0,'location_lat'],df.loc[0,'location_long']], zoom=15, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[df.loc[0,'location_lat'],df.loc[0,'location_long']], children=[
                dl.Tooltip(df.iloc[0,4]),
                dl.Popup([
                    html.H1(str(df.loc[0,'name'])),
                    html.P(df.iloc[1,9])
                ])
            ])
        ])
    ]   

app